In [254]:
# Import libaries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
import requests
import re

In [180]:
stuff = soup(requests.get('http://chubbygrub.com/restaurants/aw-restaurants/').text, 'lxml')
things = stuff.find_all('td',{'itemprop':'name'})
things

[<td itemprop="name">A&amp;W® Diet Root Beer</td>,
 <td itemprop="name">A&amp;W® Diet Root Beer</td>,
 <td itemprop="name">A&amp;W® Diet Root Beer</td>,
 <td itemprop="name">A&amp;W® Diet Root Beer</td>,
 <td itemprop="name">A&amp;W® Diet Root Beer</td>,
 <td itemprop="name">A&amp;W® Diet Root Beer</td>,
 <td itemprop="name">A&amp;W® Diet Root Beer Float (large)</td>,
 <td itemprop="name">A&amp;W® Diet Root Beer Float (medium)</td>,
 <td itemprop="name">A&amp;W® Diet Root Beer Float (small)</td>,
 <td itemprop="name">A&amp;W® Diet Root Beer Freeze (large)</td>,
 <td itemprop="name">A&amp;W® Diet Root Beer Freeze (medium)</td>,
 <td itemprop="name">A&amp;W® Diet Root Beer Freeze (small)</td>,
 <td itemprop="name">A&amp;W® Root Beer</td>,
 <td itemprop="name">A&amp;W® Root Beer</td>,
 <td itemprop="name">A&amp;W® Root Beer</td>,
 <td itemprop="name">A&amp;W® Root Beer</td>,
 <td itemprop="name">A&amp;W® Root Beer</td>,
 <td itemprop="name">A&amp;W® Root Beer</td>,
 <td itemprop="name">A&

### Step 1: Create a soup object from the home page

In [181]:
url = "http://chubbygrub.com/"

In [182]:
response = requests.get(url)
print(response.status_code)
html = response.text

200


In [183]:
chubsoup = soup(html, 'lxml')

### Step 2: Scrape the home page soup for every restaurant

Note: Your best bet is to create a list of dictionaries, one for each restaurant. Each dictionary contains the restaurant's name and slug. The result of your scrape should look something like this:

```python
restaurants = [
    {'name': 'A&W Restaurants', 'slug': 'aw-restaurants'}, 
    {'name': "Applebee's", 'slug': 'applebees'},
    ...
]
```

In [184]:
rdata = []
for x in chubsoup.find_all('a', {'class': 'btn'}):
    name = {}
    name['name'] = x.text
    name['slug'] = x.get('href')[13:]
    rdata.append(name)
rdata

[{'name': 'A&W Restaurants', 'slug': 'aw-restaurants'},
 {'name': "Applebee's", 'slug': 'applebees'},
 {'name': "Arby's", 'slug': 'arbys'},
 {'name': 'Atlanta Bread Company', 'slug': 'atlanta-bread-company'},
 {'name': "Bojangle's Famous Chicken 'n Biscuits",
  'slug': 'bojangles-famous-chicken-n-biscuits'},
 {'name': 'Buffalo Wild Wings', 'slug': 'buffalo-wild-wings'},
 {'name': 'Burger King', 'slug': 'burger-king'},
 {'name': "Captain D's", 'slug': 'captain-ds'},
 {'name': "Carl's Jr.", 'slug': 'carls-jr'},
 {'name': "Charley's Grilled Subs", 'slug': 'charleys-grilled-subs'},
 {'name': 'Chick-fil-A', 'slug': 'chick-fil-a'},
 {'name': "Chili's", 'slug': 'chilis'},
 {'name': 'Chipotle Mexican Grill', 'slug': 'chipotle-mexican-grill'},
 {'name': "Church's", 'slug': 'churchs'},
 {'name': 'Corner Bakery Cafe', 'slug': 'corner-bakery-cafe'},
 {'name': 'Dairy Queen', 'slug': 'dairy-queen'},
 {'name': "Denny's", 'slug': 'dennys'},
 {'name': 'El Pollo Loco', 'slug': 'el-pollo-loco'},
 {'name'

### Step 3: Using the slug, scrape each restaurant's page and create a single list of food dictionaries.

Your list of foods should look something like this:
```python
foods = [
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    ...
]
```

**Note**: Remove extra white space from each category

In [259]:
url=[]
baselink = "http://chubbygrub.com/restaurants/"
for x in rdata:
    for k,v in x.items():
        if k == 'slug':
            url.append(baselink+v)
url = url

In [260]:
#soup(requests.get('http://chubbygrub.com/restaurants/aw-restaurants/').text, 'lxml')
fullmenu = []

for weblink in url:
    for x in soup(requests.get(weblink).text, 'lxml').find_all('tr', {'itemtype' : "http://schema.org/NutritionInformation"}): 
        menu ={}
        menu['name'] = x.find('td',{'itemprop':'name'}).text
        menu['calories'] = x.find('td',{'itemprop':'calories'}).text
        menu['fat'] = x.find('td',{'itemprop':'fatContent'}).text
        menu['carbs'] = x.find('td',{'itemprop':'carbohydrateContent'}).text
        menu['resturant'] = re.sub('http://chubbygrub.com/restaurants/', '', weblink)
        fullmenu.append(menu)

### Step 4: Create a pandas DataFrame from your list of foods

**Note**: Your DataFrame should have 4,977 rows

In [261]:
pd.DataFrame(fullmenu)

,calories,carbs,fat,name,resturant
0,0,0,0,A&W® Diet Root Beer,aw-restaurants
1,0,0,0,A&W® Diet Root Beer,aw-restaurants
2,0,0,0,A&W® Diet Root Beer,aw-restaurants
3,0,0,0,A&W® Diet Root Beer,aw-restaurants
4,0,0,0,A&W® Diet Root Beer,aw-restaurants
5,0,0,0,A&W® Diet Root Beer,aw-restaurants
6,350,60,10,A&W® Diet Root Beer Float (large),aw-restaurants
7,170,30,5,A&W® Diet Root Beer Float (medium),aw-restaurants
8,170,30,5,A&W® Diet Root Beer Float (small),aw-restaurants
9,600,92,18,A&W® Diet Root Beer Freeze (large),aw-restaurants


### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [263]:
pd.DataFrame(fullmenu).to_csv('DaMenu.csv')